In [1]:
import pdfplumber
import pandas as pd
from deep_translator import GoogleTranslator
import PyPDF2
import re
import operator
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
    
pdf = pdfplumber.open("france.pdf")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lafeizhuangyuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lafeizhuangyuan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def keywords():
    #this is not the whole list of the keywords
    questions_to_keywords={
    "holiday":"Can your whole household afford to go for a week’s annual holiday, away from home?",
    "vegetarian":"Can your household afford a meal with meat, chicken, fish(or vegetarian equivalent)?",
    "expense": "Can your household afford an unexpected required expense(amount to be filled) and pay through its own resources?",
    "telephone":"Does your household have a telephone(fixed landline or mobile)?",
    "colour TV":"Does your household have a color TV?",
    "washing machine":"Does the household have a washing machine? ",
    "van":"Does your household have a car/van for private use? ",
    "dwelling":"Do you have any of the following problems with your dwelling/accommodation? ",
    "warm":"Can your household afford to keep its home adequately warm?  "
} 
    
    translated_keywords = []
    for key in questions_to_keywords.keys():
        translated = GoogleTranslator(source='en', target='french').translate(key)
        translated_keywords.append(translated)
    return translated_keywords

    
    
translated_keywords = keywords()
print(translated_keywords)



['vacances', 'végétarien', 'frais', 'Téléphone', 'téléviseur couleur', 'Machine à laver', 'van', 'logement', 'chaleureux']


# Text Preprocessing functions 👇

In [13]:
#testing regex - re.IGNORECASE works so it should work when matching keywords but doesnt for some reason
def test_regex():
    matched = []
    matched.append(re.findall('hé',"héllo"))
    return matched


                               
#function to translate from detected language into english using deep translator library 
def translator(lines):
    translated_array =[]
    for i in lines:
        to_translate = i
        translated = GoogleTranslator(source='auto', target='en').translate(i)
        translated_array.append(translated)
    return translated_array


#cleans text removing next lines, whitespace,punctuation,setc 
def clean(text):
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    #removing punctuations'
    text = re.sub(r'[^\w\s]','',text)
    return text

def tokenize(text) :
    return nltk.word_tokenize(text)

 **Stop words/(stemming)/ Lemmatization function**

In [14]:
nltk.download('stopwords')
stop_words= stopwords.words('english') # can be changed to ('french')
#stop_words.append('unimportant word')
#print (stop_words)

#stemmer works not good
#from nltk.stem import PorterStemmer#WordNetLemmatizer
#stemmer = PorterStemmer()
#input_str = "I loved apples"
#print(stemmer.stem(input_str))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
#input_str = "I loved apples"
#for word in input_str:
    #print(lemmatizer.lemmatize(word))
#print(lemmatizer.lemmatize(input_str))



    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lafeizhuangyuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# the main functions

In [18]:
#DO NOT CHANGE- FULLY IDENTIFIES QUESTIONS ON A PAGE
#given a page of a docuent or list of pages, will find all questions on that page and translate into english 
def translate_document(pages):
    pdf1 = pdfplumber.open("france.pdf")
    translated_array = []
    #pages = list(pages)
    # writing page 161 will translate page 162
    pages= pages
    for number in pages:
        p1 = pdf1.pages[number]
        im = p1.to_image()
        text = p1.extract_text()
        text = clean(text)
        #text = re.split('[?]',text)  
        text = re.findall('(?<=[\?\.\!]\s)[^\?\n\.]+?\?',text)
        clean_sent  = []
        for sent in text:
            clean_sent.append(sent) 
        translated_array.append(clean_sent)
        translated_array.append(translator(clean_sent))
    return translated_array



pages = []
for word in translated_keywords:
    word = word.lower()
    pdf = PyPDF2.PdfFileReader(r"france.pdf")
    for i in range(0,pdf.getNumPages()):
        PageObj = pdf.getPage(i)
        Text = PageObj.extractText()     
        if re.findall(word,Text,re.IGNORECASE):
            #pages.append(word)
            pages.append(i) 
            
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list


d = {"col1": translate_document(pages)}
DftranslatedDoc=pd.DataFrame(data =d)
DftranslatedDoc

,col1
0,[]
1,[]
2,[]
3,[]
4,[]
...,...
311,[]
312,[]
313,[]
314,[]


## Preprocessing used on  our dataframe

In [21]:

lemmatizier= WordNetLemmatizer()
for index,row in DftranslatedDoc.iterows():
    filttered_sentence=[]
    sentence = row['col1']
    sentence = re.sub(r'[^\w\s]','',sentence)
    words=nltk.word_tokenize(sentence)
    words=[ w for rw in words if not w in stop_words]
    for word in words:
        filttered_sentence.append(lemmatizer.lemmatize(word))
    #print(filttered_sentence)
    data.ix[index,'col1'] = filttered_sentence
        

      

IndentationError: unexpected indent (3841773175.py, line 3)